In [1]:
%matplotlib inline
import d2l
from mxnet import gluon, init, npx
from mxnet.gluon import nn
import pandas as pd; pd.__version__
npx.set_np()

In [2]:
def load_data_mnist(batch_size, resize=None):
    dataset = gluon.data.vision
    trans = [dataset.transforms.Resize(resize)] if resize else []
    trans.append(dataset.transforms.ToTensor())
    trans = dataset.transforms.Compose(trans)
    mnist_train = dataset.MNIST(train=True).transform_first(trans, lazy=False)
    mnist_test = dataset.MNIST(train=False).transform_first(trans, lazy=False)
    return (gluon.data.DataLoader(mnist_train, batch_size, shuffle=True,
                                  num_workers=d2l.get_dataloader_workers()),
            gluon.data.DataLoader(mnist_test, batch_size, shuffle=False,
                                  num_workers=d2l.get_dataloader_workers()))


In [3]:
batch_size = 256
train_iter, test_iter = load_data_mnist(batch_size)

In [4]:
net = nn.Sequential()
net.add(
    nn.Dense(256, activation='relu'),
    nn.Dense(10)
)
net.initialize(init.Normal(sigma=0.01))

In [5]:
loss = gluon.loss.SoftmaxCrossEntropyLoss()

In [6]:
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.1})

In [7]:
def train_epoch(net, train_iter, loss, updater):
    metric = d2l.Accumulator(3)  # train_loss_sum, train_acc_sum, num_examples
    if isinstance(updater, gluon.Trainer):
        updater = updater.step
    for X, y in train_iter:
        # Compute gradients and update parameters
        with d2l.autograd.record():
            y_hat = net(X)
            l = loss(y_hat, y)
        l.backward()
        updater(X.shape[0])
        metric.add(float(l.sum()), d2l.accuracy(y_hat, y), y.size)
    # Return training loss and training accuracy
    return metric[0]/metric[2], metric[1]/metric[2]

In [8]:
def train_model(net, train_iter, test_iter, loss, num_epochs, updater):
#     animator = d2l.Animator(xlabel='epoch', xlim=[1, num_epochs],
#                         ylim=[0.0, 1.0],
#                         legend=['train loss', 'train acc', 'test acc'])
    for epoch in range(num_epochs):
        train_metrics = train_epoch(net, train_iter, loss, updater)
        test_acc = d2l.evaluate_accuracy(net, test_iter)
        print(f'iter: {epoch} Test Accuracy: {test_acc}')
        #animator.add(epoch+1, train_metrics+(test_acc,))

In [9]:
num_epochs = 20
train_model(net, train_iter, test_iter, loss, num_epochs, trainer)

iter: 0 Test Accuracy: 0.8868
iter: 1 Test Accuracy: 0.9102
iter: 2 Test Accuracy: 0.9183
iter: 3 Test Accuracy: 0.9233
iter: 4 Test Accuracy: 0.9325
iter: 5 Test Accuracy: 0.9383
iter: 6 Test Accuracy: 0.9413
iter: 7 Test Accuracy: 0.9449
iter: 8 Test Accuracy: 0.9477
iter: 9 Test Accuracy: 0.948
iter: 10 Test Accuracy: 0.9535
iter: 11 Test Accuracy: 0.9558
iter: 12 Test Accuracy: 0.9581
iter: 13 Test Accuracy: 0.9586
iter: 14 Test Accuracy: 0.9618
iter: 15 Test Accuracy: 0.9636
iter: 16 Test Accuracy: 0.9644
iter: 17 Test Accuracy: 0.9646
iter: 18 Test Accuracy: 0.9665
iter: 19 Test Accuracy: 0.9675
